In [83]:
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
from tensorflow import keras
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential, layers
import os
import pandas as pd


In [84]:
clean_file_names = [x for x in os.listdir('code/practice_images/') if x.endswith('.jpg')]

In [85]:
wind_strength = []

for i in clean_file_names:
    wind_strength.append(i.split('_')[6].strip('.jpg'))

In [86]:
sin = []

for i in clean_file_names:
    sin.append(i.split('_')[4].strip('jpg'))

In [87]:
cos = []

for i in clean_file_names:
    cos.append(i.split('_')[4].strip('jpg'))

In [98]:
df = pd.DataFrame()

df['filename'] = clean_file_names
df['wind_strength'] = wind_strength
df['sin'] = sin
df['cos'] = cos

In [99]:
df['wind_strength'] = df['wind_strength'].apply(float)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   filename       12 non-null     object 
 1   wind_strength  12 non-null     float64
 2   sin            12 non-null     object 
 3   cos            12 non-null     object 
dtypes: float64(1), object(3)
memory usage: 512.0+ bytes


In [70]:
clean_file_names

['32.96499862670898_-106.34500274658203_02_CW000_0.07893691956996918_0.7222934365272522_1.2358866930007935.jpg',
 '32.914998626708986_-106.39500274658204_00_CW000_0.07893691956996918_0.7222934365272522_1.2358866930007935.jpg',
 '20.934999847412108_52.69499816894531_02_CW000_-0.3862479329109192_-0.49399876594543457_1.6461933851242065.jpg',
 '20.88499984741211_52.69499816894531_03_CW000_-0.3862479329109192_-0.49399876594543457_1.6461933851242065.jpg',
 '40.93499984741211_101.70500030517579_02_CW000_0.17025038599967957_0.3836674392223358_1.167919635772705.jpg',
 '40.88499984741211_101.65500030517578_00_CW000_0.17025038599967957_0.3836674392223358_1.167919635772705.jpg',
 '32.914998626708986_-106.34500274658203_03_CW000_0.07893691956996918_0.7222934365272522_1.2358866930007935.jpg',
 '40.93499984741211_101.65500030517578_01_CW000_0.17025038599967957_0.3836674392223358_1.167919635772705.jpg',
 '20.88499984741211_52.644998168945314_00_CW000_-0.3862479329109192_-0.49399876594543457_1.64619338

In [11]:
augmented_data = ImageDataGenerator(rescale=1./255)
                                    #brightness_range = [0.5,0.8])


In [42]:
aug_generator = augmented_data.flow_from_dataframe(dataframe=df,
                                                    directory="code/practice_images/",
                                                    x_col="filename",
                                                    y_col="target",
                                                    batch_size=1,
                                                    class_mode= 'raw',
                                                   save_to_dir='code', save_prefix='..',
                                                    save_format='png',
                                                    color_mode="grayscale",
                                                    target_size=(512, 512))

Found 12 validated image filenames.


In [43]:
def load_resnet():
    model = keras.applications.ResNet50(
                                            include_top=False, weights=None,
                                            input_shape=(512, 512,1), pooling="max",
    )
    model.trainable = True 
    model = keras.models.Sequential([ 
                            model,
                            keras.layers.Dense(1, activation='relu')
                                ])
    opt = Adam(learning_rate=0.001)
    model.compile(loss='mae',
                  optimizer=opt,
                  metrics=['mae'])
    return model
model = load_resnet()
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23581440  
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 2049      
Total params: 23,583,489
Trainable params: 23,530,369
Non-trainable params: 53,120
_________________________________________________________________


In [45]:
history = model.fit(aug_generator, epochs=1)

12/12 [==============================] - 28s 2s/step - loss: 0.6929


In [ ]:
# 'model = Sequential()
# model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=(512,512,1)))
# model.add(layers.Conv2D(32, (3,3), activation='relu'))
# model.add(layers.Conv2D(64, (3,3), activation='relu'))
# model.add(layers.MaxPool2D(2,2))
# model.add(layers.Flatten())
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.summary()        

In [ ]:
!ls code